In [31]:
import findspark
findspark.init()

In [34]:
from pyspark import SparkContext
sc = SparkContext('local[2]','estimating_salaries')

In [35]:
# sc.stop()

In [36]:
workersRDD = sc.textFile("workers.txt")

In [37]:
workersRDD

workers.txt MapPartitionsRDD[1] at textFile at <unknown>:0

In [38]:
workersRDD.take(5)

['sirano,isim,yas,meslek,sehir,aylik_gelir',
 '1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200',
 '3,Timur,30,Müzisyen,Istanbul,9000',
 '4,Burcu,29,Pazarlamaci,Ankara,4200']

In [39]:
 workersRDD = workersRDD.filter(lambda x : 'sirano' not in x)

In [40]:
workersRDD.take(5)

['1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200',
 '3,Timur,30,Müzisyen,Istanbul,9000',
 '4,Burcu,29,Pazarlamaci,Ankara,4200',
 '5,Yasemin,23,Pazarlamaci,Bursa,4800']

In [53]:
def profession_salary(x):
    profession = x.split(',')[3]
    salary = float(x.split(',')[5])
    
    return (profession, salary)

In [55]:
profession_salary_pairRDD = workersRDD.map(profession_salary)
profession_salary_pairRDD.take(5)

[('Isci', 3500.0),
 ('Memur', 4200.0),
 ('Müzisyen', 9000.0),
 ('Pazarlamaci', 4200.0),
 ('Pazarlamaci', 4800.0)]

In [56]:
profession_salary_pairRDD = profession_salary_pairRDD.mapValues(lambda x : (x,1))
profession_salary_pairRDD.take(5)

[('Isci', (3500.0, 1)),
 ('Memur', (4200.0, 1)),
 ('Müzisyen', (9000.0, 1)),
 ('Pazarlamaci', (4200.0, 1)),
 ('Pazarlamaci', (4800.0, 1))]

In [59]:
profession_salary_pairRDD = profession_salary_pairRDD.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
profession_salary_pairRDD.take(10)                                                                

[('Memur', (12200.0, 3)),
 ('Pazarlamaci', (16300.0, 3)),
 ('Tuhafiyeci', (4800.0, 1)),
 ('Tornacı', (4200.0, 1)),
 ('Isci', (3500.0, 1)),
 ('Müzisyen', (29700.0, 3)),
 ('Doktor', (32250.0, 2)),
 ('Berber', (12000.0, 1))]

In [63]:
profession_salary_pairRDD = profession_salary_pairRDD.mapValues(lambda x : x[0]/x[1])
profession_salary_pairRDD.take(8)

[('Memur', 4066.6666666666665),
 ('Pazarlamaci', 5433.333333333333),
 ('Tuhafiyeci', 4800.0),
 ('Tornacı', 4200.0),
 ('Isci', 3500.0),
 ('Müzisyen', 9900.0),
 ('Doktor', 16125.0),
 ('Berber', 12000.0)]